In [4]:
# Cluster and Topic Modeling exploration
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/clustering/agglomerative.py

from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd

In [5]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
tob = pd.read_csv('./tob_v1_classified.csv', index_col='index')

tob.head()

,user,question,answer,question_class
index,,,,
0,100,hi,"Hello, how are you?",Greet
1,100,Simone,"Sorry, I still can't answer that.. Could you a...",Statement
2,100,I am Simone,"Sorry, I still can't answer that.. Could you a...",Statement
3,100,what is software testing,Software testing is an activity of software en...,whQuestion
4,100,what is structural testing,Structural testing (or White-Box testing) is a...,whQuestion


In [10]:
embedings = embedder.encode(tob['question'].values)
embedings

array([[-0.09047619,  0.04043957,  0.02390568, ...,  0.06815036,
         0.01786567,  0.01127626],
       [-0.08309868, -0.0599528 ,  0.01865478, ..., -0.03056165,
        -0.0363641 , -0.15343234],
       [-0.03010819, -0.04681816,  0.07195324, ..., -0.01680324,
        -0.05603674, -0.19724695],
       ...,
       [-0.05213145,  0.01008251, -0.09586437, ...,  0.02771363,
        -0.02413042, -0.10155366],
       [-0.09047616,  0.04043955,  0.02390566, ...,  0.06815033,
         0.01786565,  0.01127623],
       [ 0.01000331, -0.06694853, -0.02342124, ...,  0.04776511,
        -0.03691382,  0.02372807]], dtype=float32)

In [13]:
# Normalize the embeddings to unit length
embedings = embedings /  np.linalg.norm(embedings, axis=1, keepdims=True)
embedings

array([[-0.09047619,  0.04043958,  0.02390568, ...,  0.06815037,
         0.01786567,  0.01127626],
       [-0.08309869, -0.0599528 ,  0.01865478, ..., -0.03056165,
        -0.03636411, -0.15343235],
       [-0.03010819, -0.04681816,  0.07195324, ..., -0.01680324,
        -0.05603674, -0.19724695],
       ...,
       [-0.05213145,  0.01008251, -0.09586437, ...,  0.02771363,
        -0.02413042, -0.10155366],
       [-0.09047617,  0.04043956,  0.02390566, ...,  0.06815033,
         0.01786566,  0.01127623],
       [ 0.01000331, -0.06694853, -0.02342124, ...,  0.04776511,
        -0.03691382,  0.02372807]], dtype=float32)

In [14]:
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embedings)
cluster_assignment = clustering_model.labels_

cluster_assignment

array([196, 235, 235, ...,   7, 196, 144])

In [15]:
tob['question_cluster'] = cluster_assignment

In [16]:
tob.head()

,user,question,answer,question_class,question_cluster
index,,,,,
0,100,hi,"Hello, how are you?",Greet,196
1,100,Simone,"Sorry, I still can't answer that.. Could you a...",Statement,235
2,100,I am Simone,"Sorry, I still can't answer that.. Could you a...",Statement,235
3,100,what is software testing,Software testing is an activity of software en...,whQuestion,118
4,100,what is structural testing,Structural testing (or White-Box testing) is a...,whQuestion,23


In [17]:
tob.value_counts('question_cluster')

question_cluster
256    164
196    107
118     75
199     71
162     69
      ... 
230      2
240      2
246      2
109      2
231      2
Name: count, Length: 267, dtype: int64

In [44]:
tob[tob['question_cluster'] == 196]

,user,question,answer,question_class,question_cluster
index,,,,,
0,100,hi,"Hello, how are you?",Greet,196
27,100,Hi,"Hello, ask me a question!",Greet,196
31,101,hi,"Hi there, what do you need?",Greet,196
45,102,Hi,Welcome!,Greet,196
47,103,hi,Welcome!,Greet,196
...,...,...,...,...,...
2675,91,Hi,I don't know how to respond this,Greet,196
2678,92,Hi,I don't know how to respond this,Greet,196
2697,93,hi,Hi there,Greet,196


In [40]:
tob12 = pd.read_csv('./tob_v1_x_v2.csv', index_col='index')
tob12

,user,question,question_class,answer_v1,answer_v2,v1_x_v2
index,,,,,,
0,100,hi,Greet,"Hello, how are you?",Hi,0.618338
1,100,Simone,Statement,"Sorry, I still can't answer that.. Could you a...",Hi,0.117703
2,100,I am Simone,Statement,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't understand your question, coul...",0.575173
3,100,what is software testing,whQuestion,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
4,100,what is structural testing,whQuestion,Structural testing (or White-Box testing) is a...,Structural testing (or White-Box testing) is a...,1.000000
...,...,...,...,...,...,...
2984,98,what is the definition of state transition tes...,whQuestion,"It is a criteria within functional testing, bu...",I don't understand what you mean,0.136184
2985,98,what is graph testing,whQuestion,Software testing is an activity of software en...,I don't understand what you mean,-0.001124
2986,98,what is graph matrix testing,whQuestion,"It is a criteria within functional testing, bu...",Software testing is an activity of software en...,0.409774


In [41]:
tob12 = tob12.join(tob['question_cluster'], on='index', how='inner')

tob12 = tob12[["user", "question", "question_class", "question_cluster", "answer_v1", "answer_v2", "v1_x_v2"]]
tob12 = tob12.rename({"v1_x_v2": "v1_v2_similarity"})

In [42]:
tob12.to_csv('./tob_question_cluster.csv')